In [1]:
import sys
sys.path.insert(0, '../')
import myTA
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from strategies import Backtesting, MACDZeroHistPrediction, makeAnalysis, SharpeRatio, get_df, init_backtesting_results_file, get_allowedIntervals, get_parameter_combination_number

c:\Users\QbaSo\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
sharpe_ratio = SharpeRatio()

file_name = "backtest_resluts/backtesting_results_top1_grid_search.csv"
init_backtesting_results_file(file_name)
allowedIntervals = get_allowedIntervals().iloc[2:4]

tickers = pd.read_csv("../nasdaqTickers.csv")
top_100_tickers = tickers.sort_values('Market Cap', ascending=False).head(20)['Symbol'].values

In [3]:
tickers = top_100_tickers

linerPredictionWindows = [4,5]
fastperiods = [11,12,13]
slowperiods = [25,26,27]
signalperiods = [9]

# linerPredictionWindows = np.arange(2,6)
# fastperiods = np.arange(8,18)
# slowperiods = np.arange(20,30)
# signalperiods = np.arange(5,13)

df = pd.DataFrame()

get_parameter_combination_number(tickers, linerPredictionWindows, fastperiods, slowperiods, signalperiods, allowedIntervals)
for i in range(len(allowedIntervals)):
    end = dt.datetime.now()
    start = end - allowedIntervals.iloc[i]['back_allowed']
    interval = allowedIntervals.iloc[i]['intervals']

    for ticker in tickers:
        data = yf.download(ticker, start=start, end=end, interval=interval, progress=False)

        for linerPredictionWindow in linerPredictionWindows:
            for fastperiod in fastperiods:
                for slowperiod in slowperiods:
                    for signalperiod in signalperiods:
                        strategy = MACDZeroHistPrediction(linerPredictionWindow, ticker, start, end, interval, fastperiod, slowperiod, signalperiod)
                        try:
                            data = makeAnalysis(strategy, data)
                        except:
                            pass
                            continue
                        strategy.getBackTestSignals(data)

                        bt = Backtesting(10000, strategy)
                        bt.calculateProfit()
                        df = get_df(df, bt, sharpe_ratio)

    df.to_csv(file_name, mode='a', index=False, header=False)
    df = pd.DataFrame()

backtest will run for: 720 pameters combination

1 Failed download:
- BRK/A: No data found for this date range, symbol may be delisted

1 Failed download:
- BRK/B: No data found for this date range, symbol may be delisted

1 Failed download:
- BRK/A: No data found for this date range, symbol may be delisted

1 Failed download:
- BRK/B: No data found for this date range, symbol may be delisted


In [20]:
results = pd.read_csv(file_name)
results.groupby(['interval','linerPredictionWindow','fastperiod','slowperiod','signalperiod']).mean()

budget_start  \
interval linerPredictionWindow fastperiod slowperiod signalperiod                 
15m      4                     11         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
                               12         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
                               13         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
         5                     11         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
                               12         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
                               13         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
5m       4                     11         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
                               12         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
                               13         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
         5                     11         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
                               12         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   
                               13         25         9                  10000.0   
                                          26         9                  10000.0   
                                          27         9                  10000.0   

                                                                      result  \
interval linerPredictionWindow fastperiod slowperiod signalperiod              
15m      4                     11         25         9              5.166667   
                                          26         9             13.944444   
                                          27         9             10.222222   
                               12         25         9             14.500000   
                                          26         9             11.333333   
                                          27         9             11.333333   
                               13         25         9             10.944444   
                                          26         9             24.166667   
                                          27         9             13.222222   
         5                     1

In [16]:
results = pd.read_csv('backtest_resluts/backtesting_results_top20_grid_search_MACDZeroHistPredictionAndSMA.csv')
results.groupby(['interval','linerPredictionWindow','fastperiod','slowperiod','signalperiod']).mean().sort_values('sharpe_ratio', ascending=False)

budget_start  \
interval linerPredictionWindow fastperiod slowperiod signalperiod                 
15m      4                     12         25         9                  10000.0   
5m       5                     11         25         9                  10000.0   
15m      4                     11         26         9                  10000.0   
5m       5                     12         27         9                  10000.0   
                               11         26         9                  10000.0   
                               12         25         9                  10000.0   
         4                     11         25         9                  10000.0   
                               12         27         9                  10000.0   
15m      5                     13         27         9                  10000.0   
5m       4                     11         26         9                  10000.0   
                                          27         9                  10000.0   
15m      4                     13         25         9                  10000.0   
5m       5                     12         26         9                  10000.0   
         4                     13         26         9                  10000.0   
15m      5                     13         25         9                  10000.0   
                               12         27         9                  10000.0   
         4                     11         25         9                  10000.0   
5m       4                     12         26         9                  10000.0   
         5                     13         27         9                  10000.0   
         4                     13         25         9                  10000.0   
15m      4                     12         26         9                  10000.0   
         5                     11         26         9                  10000.0   
                               12         26         9                  10000.0   
5m       5                     13         26         9                  10000.0   
15m      5                     13         26         9                  10000.0   
         4                     11         27         9                  10000.0   
         5                     11         25         9                  10000.0   
         4                     13         27         9                  10000.0   
5m       5                     13         25         9                  10000.0   
                               11         27         9                  10000.0   
15m      5                     12         25         9                  10000.0   
5m       4                     13         27         9                  10000.0   
15m      4                     12         27         9                  10000.0   
         5                     11         27         9                  10000.0   
         4                     13         26         9                  10000.0   
5m       4                     12         25         9                  10000.0   

                                                                      result  \
interval linerPredictionWindow fastperiod slowperiod signalperiod              
15m      4                     12         25         9             21.888889   
5m       5                     11         25         9             -5.666667   
15m      4                     11         26         9             25.000000   
5m       5                     12         27         9             -6.166667   
                               11         26         9             -3.888889   
                               12         25         9             -5.055556   
         4                     11         25         9             -8.944444   
                               12         27         9             -9.777778   
15m      5                     13         27         9             12.777778   
5m       4                     1